In [130]:
import requests as r
from bs4 import BeautifulSoup as bs
import unicodedata
import time 
import base64

class lnn():
    def __init__(self):
        self.url='https://www.lanacion.com.ar'
   
    def get(self,url):        
        nota=r.get(url)
        sopa=bs(nota.content,features="lxml")
        self.titulo=sopa.find('h1').get_text(strip=True)
        try:
            self.bajada=sopa.find('epigrafe').get_text(strip=True)
        except:
            pass
        self.cuerpo=' '.join([unicodedata.normalize("NFKD",x.get_text()).strip() for x in sopa.find('section',{'id':'cuerpo'}).find_all('p')][:-1])
        bolds=sum([x.find_all('b') for x in sopa.find('section',{'id':'cuerpo'}).find_all('p', recursive=False)],[])
        self.bolds=[x.get_text(strip=True) for x in bolds]
        self.bold=' / '.join(self.bolds)     
        self.quotes=' / '.join([x.split('”')[0] for x in self.cuerpo.split('“')[1:]])
        token=base64.b64encode(bytes(sopa.find('div',{'id' : 'tokenLF'}).get('data-entrada'), 'utf-8')).decode('ascii')
        com=r.get('https://data.livefyre.com/bs3/v3.1/la-nacion.fyre.co/356483/'+token+'/init')
        data=com.json()
        np=data['collectionSettings']['archiveInfo']['nPages']
        comms=[]
        for n in range(0,np):
            com=r.get('https://data.livefyre.com/bs3/v3.1/la-nacion.fyre.co/356483/'+token+'/'+str(n)+'.json')
            comms.extend(com.json()['content'][:])
            time.sleep(0.2)
        coms=[]    
        for j,x in enumerate(comms):
            try:
                coms.append(bs(x['content']['bodyHtml']).get_text().strip())
            except:
                pass    
        self.coms=coms
        
    def hoy(self):
        hoy=r.get('https://www.lanacion.com.ar')
        urls=[]
        h1s=bs(hoy.content).find_all('h1')
        urls.extend([h1.find('a').get('href') for h1 in h1s])
        h2s=bs(hoy.content).find_all('h2')
        urls.extend([h2.find('a').get('href') for h2 in h2s])
        headers = {
        'authority': 'www.lanacion.com.ar',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
        'accept': '*/*',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.lanacion.com.ar/',
        'accept-language': 'es-419,es;q=0.9,en;q=0.8,gl;q=0.7',   
        }

        params = (
        ('utm_source', 'navigation'),
        ('datamodule', """anexo_2;tema_1;tema_2;tema_3;opinion;
         tema_4;tema_5;tema_6;mas-leidas;comercial_1;
         tema_7;tema_8;comercial_2;tema_9;tema_10;tema_11;
         tema_12;tema_13;tema_14;tema_15;tema_16;tema_17;tema_18
         """),
        )

        r1= r.get('https://www.lanacion.com.ar/', headers=headers, params=params)
        data=r1.json()['Modules']
        urls2=[]
        for tema in data:
            urls2.extend([x.find('a').get('href') for x in bs(tema['Value']).find_all('h2')])
            try:
                urls2.extend([x.get('href') for x in bs(tema['Value']).find('ul').find_all('a')])
            except:
                pass
        urls.extend(urls2)
        urls.sort()
        self.urls=urls

def get(lista):
    notas=[]
    for url in lista:
        nota=lnn()
        try:
            nota.get(nota.url+url)
            notas.append(nota)
            time.sleep(0.2)
        except:
            try:
                nota.get(url)
                notas.append(nota)
                time.sleep(0.2)
            except:
                print(url)
    return(notas)       
            

In [131]:
portal=lnn()
portal.hoy()

In [132]:
portal.urls

['/buenos-aires/que-son-ciudades-15-minutos-como-buenos-nid2533196',
 '/deportes/floyd-mayweather-volvera-al-ring-enfrentar-youtuber-nid2533116',
 '/deportes/futbol/benedetto-gring-disculpas-nid2534069',
 '/deportes/lionel-messi-vs-cristiano-ronaldo-estadisticas-rivalidad-nid2528016',
 '/deportes/rugby/la-world-rugby-tambien-critico-mensajes-racistas-nid2534073',
 '/deportes/tenis/del-potro-esta-brasil-someterse-tratamiento-celulas-nid2534118',
 '/deportes/un-jeque-arabe-compro-mitad-del-equipo-nid2534104',
 '/economia/campo/arandanos-reconocida-variedad-trajo-al-pais-firma-nid2528115',
 '/economia/campo/trigo-cultivo-se-enfrenta-falta-agua-heladas-nid2530121',
 '/economia/el-economista-roberto-cachanosky-conto-le-ofrecieron-nid2533497',
 '/economia/emmanuel-alvarez-agis-la-unica-fuente-disponible-de-financiamiento-en-dolares-para-la-argentina-es-el-fmi-nid2530486',
 '/economia/jubilaciones-advierten-nueva-ley-se-perdera-aumento-nid2530077',
 '/economia/negocios/facturan-us23-millones-

In [133]:
hoy=get(portal.urls)

/lifestyle/christmasworld-ideas-deco-simples-feria-navidad-mas-nid2530071
/sociedad/innovacion-medicamentos-se-ubican-centro-escena-nid2529979
https://www.lanacion.com.ar/tecnologia/aevum-ravn-x-el-avion-autonomo-que-pondra-satelites-en-orbita-nid08122020/
https://www.lanacion.com.ar/tecnologia/playstation-5-estos-son-los-juegos-que-debes-tener-para-aprovechar-sus-graficos-nid08122020/
https://www.lanacion.com.ar/turismo/cayote-alfajor-y-postre-balcarce-una-vuelta-por-la-historia-de-los-mejores-dulces-autoctonos-nid04122020/


['En serio? En esto termina el "Floyd Gayweather"? Patetico...']

In [9]:
url='https://www.lanacion.com.ar/el-mundo/el-ranking-mejores-paises-pasar-pandemia-deja-nid2519658'
url='https://www.lanacion.com.ar/economia/porque-bonos-argentinos-estan-excluidos-nueva-primavera-nid2519696'
url='https://www.lanacion.com.ar/seguridad/puerto-madero-mataron-disparo-extranjero-robarle-bicicleta-nid2534082'

In [60]:
nota=lnn()
nota.get(url)

In [40]:
nota.titulo

'Retiro: mataron de un balazo en el cuello a un extranjero para robarle la bicicleta'

In [12]:
nota.bajada

'Por el crimen detuvieron a un menor de 15 años'

In [13]:
nota.quotes

''

In [14]:
nota.bolds

['de nacionalidad armenia fue asesinado de un balazo en el cuello',
 'fue detenido, armado, un adolescente de 15 años',
 'límite entre Retiro y Puerto Madero',
 'Dimitri Amiryan,',
 'Hospital Fernández.',
 'LA NACION,',
 'Dimitri',
 'residía en el país desde hace 20 años.',
 'atrapó al joven a 500 metros del lugar,',
 'LA NACION',
 "Marcelo D'Alessandro.",
 'Alberto Crescenti.',
 'una herida de arma de fuego en la región facial"',
 'Se lo trató de compensar, se lo trasladó al Hospital Fernández y, lamentablemente, me acaban de informar que falleció']

'Este es el resultado del "Estado presente" y los planes sociales...'

In [43]:
nota.comm

1574

In [44]:
coms=[]    
for j,x in enumerate(nota.comm):
    try:
        coms.extend(bs(x['content']['bodyHtml']).get_text().strip())
    except:
        pass    

In [55]:
a=[]

In [57]:
a.append(bs(nota.comm[0]['content']['bodyHtml']).get_text().strip())

In [58]:
a

['Villa 31... O de la Rodrigo Bueno...??']

In [169]:
nota=r.get(url)
sopa=bs(nota.content,features="lxml")

In [170]:
' '.join([unicodedata.normalize("NFKD",x.get_text()).strip() for x in sopa.find('section',{'id':'cuerpo'}).find_all('p', recursive=False)])

'Uno de los hermanos que eran juzgados en la ciudad de Salta por el homicidio del turista francés Mathieu Martin, de 32 años, que en agosto de 2018 desapareció cuando se dirigía como mochilero a la localidad de Iruya, fue condenado a 13 años de prisión, mientras que su hermano fue absuelto por el beneficio de la duda. Los jueces de la Sala VI del Tribunal del Juicio de Salta, Guillermo Pereyra, Mónica Mukdsi y José Luis Riera, condenaron esta tarde a Juan Cuevas, de 39 años, a la pena 13 años de prisión efectiva por resultar autor penalmente responsable del delito de homicidio simple en perjuicio de Martin. Además, dispusieron su traslado a la Unidad Carcelaria 1 de Salta y la extracción de material genético por parte del Servicio de Biología Molecular del Departamento Técnico del Cuerpo de Investigaciones Fiscales (CIF), para su incorporación al Banco de Datos Genéticos. En tanto, su hermano Froilán Cuevas, de 41 años, fue absuelto por el beneficio de la duda. Los 

In [171]:
[x.find_all('b') for x in sopa.find('section',{'id':'cuerpo'}).find_all('p', recursive=False)]

[[<b> turista francés Mathieu Martin</b>,
  <b>fue condenado a 13 años de prisión</b>],
 [],
 [],
 [],
 [],
 [],
 [<b>desapareció en agosto de 2018, cuando viajaba desde Humahuaca a la localidad salteña de Iruya</b>],
 [],
 [],
 [],
 [],
 [],
 [],
 [<b> la querella, a cargo de Matías Adet no formuló pedido de pena por requerimiento de los familiares, que escribieron una carta en la que solicitaron saber la verdad y hallar el cuerpo de Martin.</b>],
 [],
 [],
 [<b> aseguraron no ser los autores del hecho</b>,
  <b> afirmaron que ambos fueron obligados a confesar el crimen luego de sufrir maltratos de la policía</b>],
 [<b>Mathieu Pierre Martin desapareció en agosto de 2018, cuando recorría el norte argentino como mochilero</b>],
 [<b>su cuerpo nunca fue hallado.</b>]]

## Comentarios

In [188]:
import base64
token=base64.b64encode(bytes(sopa.find('div',{'id' : 'tokenLF'}).get('data-entrada'), 'utf-8')).decode('ascii')

In [189]:
com=r.get('https://data.livefyre.com/bs3/v3.1/la-nacion.fyre.co/356483/'+token+'/init')
data=com.json()

In [190]:
data['collectionSettings']['archiveInfo']

{'nPages': 1,
 'pageInfo': {'0': {'url': '/la-nacion.fyre.co/356483/MTY5MjY5NQ==/0.json',
   'last': 1606271934,
   'first': 1606270635}},
 'pathBase': '/la-nacion.fyre.co/356483/MTY5MjY5NQ==/'}

In [27]:
import base64
token=base64.b64encode(bytes(sopa.find('div',{'id' : 'tokenLF'}).get('data-entrada'), 'utf-8')).decode('ascii')
com=r.get('https://data.livefyre.com/bs3/v3.1/la-nacion.fyre.co/356483/'+token+'/init')
data=com.json()
np=data['collectionSettings']['archiveInfo']['nPages']
coms=[]
for n in range(0,np):
    com=r.get('https://data.livefyre.com/bs3/v3.1/la-nacion.fyre.co/356483/'+token+'/'+str(n)+'.json')
    coms.extend(com.json()['content'][:])
    time.sleep(0.2)
    

NameError: name 'sopa' is not defined

In [32]:
for j,x in enumerate(nota.coms):
    try:
        bs(x['content']['bodyHtml']).get_text().strip()
    except:
        pass

1 {'erefs': ['OusKWgPOYjBaqossSlsPtwTvPbHkvjB81arbmR1SEUmv9KewPcGjE95jl9UHlFReEgJBmHslU8+KHN0='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879630414', 'createdAt': 1607433838, 'parentId': ''}, 'source': 5, 'type': 0, 'event': 1607433909690350}
8 {'erefs': ['3TCTZLVyvPo+kY2UeEQH8ONn2KV50L0uW1qRDXsH+veAh5/zL606kDqQuayPdQD/m023DQRA+Tc7bEk='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879630552', 'createdAt': 1607433980, 'parentId': ''}, 'source': 5, 'type': 0, 'event': 1607433980789465}
11 {'erefs': ['u73Khg4X1a1/VA1w6ecMB/N+89QfvZr/aOnx3RyGaeykE6jN5p/3Mp6uAiMn1IbUVXAX7f9pNYCmXWY='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879630608', 'createdAt': 1607434044, 'parentId': ''}, 'source': 5, 'type': 0, 'event': 1607434045124678}
12 {'erefs': ['OusKWgPOYjBaqossSlkPtgTvPbG27DUr16GClR9SQBOp86Gzb8GlHYUzl4RQkwNeRAZFwCsvVc2KTY8='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879630615', 'createdAt': 1607434050, 'parentId': ''}, 'sour

795 {'erefs': ['cYpFl4q+L2dnj6G8tKJG0gQNMHdbIgWfJ4XerLjwW7/rlY8XQxJA2eKkMBoW8KCKtHRaV5PN3z4au2c='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879635095', 'createdAt': 1607438122, 'parentId': ''}, 'source': 5, 'type': 0, 'event': 1607438188706790}
798 {'erefs': ['u73Khg4X1a1/VA1w7OIKDfN+84FO7pinOeGn0RXQPbikFqmc5JqmNJytVCMjgtaDAyFH7fY8M4ChCTU='], 'vis': 2, 'collectionId': '250152065', 'content': {'ancestorId': '879635100', 'id': '879635362', 'createdAt': 1607438346, 'parentId': '879635273'}, 'source': 5, 'type': 0, 'event': 1607438346960125}
801 {'erefs': ['KZfQ1kCts/mAwGjQkxsa2nr2NzIiJ/bW4r0yrcSjrizV8lQd8BdkH3TeUYhzGzArizWibFbiCKugg0o='], 'vis': 2, 'collectionId': '250152065', 'content': {'id': '879635112', 'createdAt': 1607438140, 'parentId': ''}, 'source': 0, 'type': 0, 'event': 1607438364358856}
802 {'vis': 0, 'collectionId': '250152065', 'content': {'id': '879635129', 'createdAt': 1607438162, 'parentId': ''}, 'source': 5, 'type': 0, 'event': 1607438208175401}
805 {'e

### hoy

In [116]:
def hoy(self):
    hoy=r.get('https://www.lanacion.com.ar')
    urls=[]
    h1s=bs(hoy.content).find_all('h1')
    urls.extend([h1.find('a').get('href') for h1 in h1s])
    h2s=bs(hoy.content).find_all('h2')
    urls.extend([h2.find('a').get('href') for h2 in h2s])
    headers = {
        'authority': 'www.lanacion.com.ar',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
        'accept': '*/*',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.lanacion.com.ar/',
        'accept-language': 'es-419,es;q=0.9,en;q=0.8,gl;q=0.7',   
    }

    params = (
        ('utm_source', 'navigation'),
        ('datamodule', """anexo_2;tema_1;tema_2;tema_3;opinion;
         tema_4;tema_5;tema_6;mas-leidas;comercial_1;
         tema_7;tema_8;comercial_2;tema_9;tema_10;tema_11;
         tema_12;tema_13;tema_14;tema_15;tema_16;tema_17;tema_18
         """),
    )

    r1= r.get('https://www.lanacion.com.ar/', headers=headers, params=params)
    data=r1.json()['Modules']
    urls2=[]
    for tema in data:
        urls2.extend([x.find('a').get('href') for x in bs(tema['Value']).find_all('h2')])
        try:
            urls2.extend([x.get('href') for x in bs(tema['Value']).find('ul').find_all('a')])
        except:
            pass
    urls.extend(urls2)
    urls.sort()
    self.portal=urls


In [115]:
urls.sort()
urls

['/buenos-aires/proyecto-colillas-cigarrillos-nid2534113',
 '/buenos-aires/que-son-ciudades-15-minutos-como-buenos-nid2533196',
 '/deportes/floyd-mayweather-volvera-al-ring-enfrentar-youtuber-nid2533116',
 '/deportes/futbol/benedetto-gring-disculpas-nid2534069',
 '/deportes/la-confesion-amigo-diego-maradona-me-dijo-nid2533421',
 '/deportes/rugby/la-world-rugby-tambien-critico-mensajes-racistas-nid2534073',
 '/deportes/tenis/del-potro-esta-brasil-someterse-tratamiento-celulas-nid2534118',
 '/deportes/un-jeque-arabe-compro-mitad-del-equipo-nid2534104',
 '/economia/campo/arandanos-reconocida-variedad-trajo-al-pais-firma-nid2528115',
 '/economia/campo/trigo-cultivo-se-enfrenta-falta-agua-heladas-nid2530121',
 '/economia/el-economista-roberto-cachanosky-conto-le-ofrecieron-nid2533497',
 '/economia/emmanuel-alvarez-agis-la-unica-fuente-disponible-de-financiamiento-en-dolares-para-la-argentina-es-el-fmi-nid2530486',
 '/economia/jubilaciones-advierten-nueva-ley-se-perdera-aumento-nid2530077',
